In [ ]:
from src.utils.get_visualize import DataVisualizer

def check_result():
    # TODO: create a function to do simple check for img
    # It should be something like:
    # Input one img, output 1 label
    DataVisualizer()
    pass